# <center>Analyzing NYC Traffic Collision Data</center>
## <center>A 15-688 Project by:</center><center><br/> Ahmet Emre Unal (ahmetemu)<br/><br/>Marco Peyrot (mpeyrotc)</center>

In [ ]:
import requests 
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import scipy
import collections

raw_data_file_name = 'NYPD_Motor_Vehicle_Collisions.csv'

In [ ]:
def load_data(file_name):
    collision = pd.read_csv(file_name, 
                            na_filter=False, 
                            parse_dates={'DATE_COMBINED' : ['DATE', 'TIME']}, 
                            infer_datetime_format=True)
    
    # Remove rows that don't have the necessary data
    columns_to_check_for_empty = ['LOCATION', 'LATITUDE', 'LONGITUDE', 'BOROUGH',
                                  'ZIP CODE', 'ON STREET NAME', 'CROSS STREET NAME', 
                                  'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 1']
    for column in columns_to_check_for_empty:
        collision = collision[collision[column] != '']
        collision = collision[collision[column] != 'UNKNOWN']

    # Drop unneeded columns
    columns_to_drop = ['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 
                       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 
                       'CONTRIBUTING FACTOR VEHICLE 5', 'LOCATION', 'UNIQUE KEY',
                       'OFF STREET NAME', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3',
                       'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']
    for column in columns_to_drop:
        collision = collision.drop(column, axis=1)
        
    # Set column types
    collision['ZIP CODE'] = collision['ZIP CODE'].astype(int)
    collision['LATITUDE'] = collision['LATITUDE'].astype(float)
    collision['LONGITUDE'] = collision['LONGITUDE'].astype(float)
    
    # Rename date column to just 'DATE'
    collision = collision.rename(columns={'DATE_COMBINED':'DATE'})
    
    # Reset index
    collision = collision.reset_index(drop=True)
    
    return collision

def create_time_features(collision):
    # Create one-hot time of day representation
    ## Date part is unimportant
    morning_rush_begin = pd.datetime(2000, 01, 01, 07, 00, 00).time()
    morning_rush_end = pd.datetime(2000, 01, 01, 11, 00, 00).time()
    evening_rush_begin = pd.datetime(2000, 01, 01, 16, 00, 00).time()
    evening_rush_end = pd.datetime(2000, 01, 01, 20, 00, 00).time()
    collision_time = collision['DATE'].dt.time
    
    ## Night Time 00:00-06:59 & 20:00-00:00
    night_time = (collision_time >= evening_rush_end) | (collision_time < morning_rush_begin)
    night_time_onehot = pd.get_dummies(night_time).loc[:, True].astype(int)
    collision = collision.assign(NIGHT_TIME = night_time_onehot.values)
    
    ## Morning Rush 07:00-10:59
    morning_rush = (collision_time >= morning_rush_begin) & (collision_time < morning_rush_end)
    morning_rush_onehot = pd.get_dummies(morning_rush).loc[:, True].astype(int)
    collision = collision.assign(MORNING_RUSH = morning_rush_onehot.values)
    
    ## Night time 00:00-06:59 & 20:00-00:00
    day_time = (collision_time >= morning_rush_end) & (collision_time < evening_rush_begin)
    day_time_onehot = pd.get_dummies(day_time).loc[:, True].astype(int)
    collision = collision.assign(DAY_TIME = day_time_onehot.values)
    
    ## Evening Rush 16:00-19:59
    evening_rush = (collision_time >= evening_rush_begin) & (collision_time < evening_rush_end)
    evening_rush_onehot = pd.get_dummies(evening_rush).loc[:, True].astype(int)
    collision = collision.assign(EVENING_RUSH = evening_rush_onehot.values)
    
    return collision

def create_date_features(collision):
    collision_day = collision['DATE'].dt.dayofweek
    is_weekday = (collision_day <= 4)
    
    weekday_onehot = pd.get_dummies(is_weekday).loc[:, True].astype(int)
    collision = collision.assign(WEEKDAY = weekday_onehot.values)
    
    weekend_onehot = pd.get_dummies(is_weekday).loc[:, False].astype(int)
    collision = collision.assign(WEEKEND = weekend_onehot.values)
    
    return collision

def create_vehicle_features(collisions):
    pass
    # Create one-hot vehicle type representation
#     vehicle_types_onehot = pd.get_dummies(collision.loc[:, 'VEHICLE TYPE CODE 1']).astype(int)

In [ ]:
# Load the collision data
collision = load_data(raw_data_file_name)
collision = create_time_features(collision)
collision = create_date_features(collision)
    
print collision.head(6)
print collision.dtypes
print len(collision)

In [ ]:
def parse_page(html):
    """
    Parse the table contained in the bycicle lanes webpage.

    Args:
        html (string): String of HTML corresponding to the data source webpage.

    Returns:
        a dictionary that contains mappings from a category to the list containing the data.
        These categories are: street, begins, ends, and borough.
    """

    # Write solution here
    soup = BeautifulSoup(html, 'html.parser')
    result = {
        'street': [],
        'begins': [],
        'ends': [],
        'borough': []
    }

    table = soup.findChildren('tbody')[0]
    rows = table.findChildren(['tr'])
    
    for row in rows:
        cells = row.findChildren('td')
        content = cells[0].text
        m = re.search(r'^([a-zA-Z\s0-9\-\.,\(\)]+) (from)*(between)* '
                      r'([a-zA-Z\s0-9\-\.,\(\)]+) (to)*(and)* '
                      r'([a-zA-Z\s0-9\-\.,\(\)]+)$', content)
        
        # content that does not follow this syntax is discarded because
        # it refers to landscapes or parks.
        if m is not None:
            result['street'].append(m.group(1).upper())
            result['begins'].append(m.group(4).upper())
            result['ends'].append(m.group(7).upper())
            result['borough'].append(cells[2].text.upper())
            
    df = pd.DataFrame(result)
    
    df.loc[df.borough == 'THE BRONX', 'borough'] = 'BRONX'
    
    df.rename(columns={'borough': 'BOROUGH', 'begins': 'BEGINS', 'ends': 'ENDS', 'street': 'ON STREET NAME'}, inplace=True)
        
    return df
    
def extract_bycicle_lanes(url):
    """
    Retrieve all of the bycicle lane information for the city of New York.

    Parameters:
        url (string): page URL corresponding to the listing of bycicle lane information.

    Returns:
        bycicle_lanes (Pandas DataFrame): list of dictionaries containing extracted lane information.
    """
    response = requests.get(url).text
    result = parse_page(response)
    
    return result

bycicle_lanes = extract_bycicle_lanes('http://www.nyc.gov/html/dot/html/bicyclists/lane-list.shtml')
print bycicle_lanes.columns

In [ ]:
def merge_bycicle_lanes(df1, df2, columns):
    """
    merge the both dataframes on the specified columns.

    Parameters:
        df1 (DataFrame): pandas dataframe containing some data.
        df2 (DataFrame): pandas dataframe containing some data.

    Returns:
        result (Pandas DataFrame): dataframe containing data from both dataframes.
    """
    result = pd.merge(df1, df2, how='left', on=columns)
    result = result.assign(HAS_LANE = [0 if x is np.nan else 1 for x in result.loc[:, 'BEGINS']])
    
    result = result.drop('BEGINS', 1)
    result = result.drop('ENDS', 1)
    
    return result.drop_duplicates()

print merge_bycicle_lanes(collision, bycicle_lanes, ['ON STREET NAME', 'BOROUGH']).head(6)